In [5]:
import pandas as pd
from ISO3166 import ISO3166
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import time
import requests

# to calculate analysis time
start_time = time.time()

# Field names for the dataset
fields_name = ['tweet_date', 'account', 'ioc_type', 'ioc_value', 'type_of_attack', 'tweet_url', 'text','mb_status', 'mb_first_seen',
               'mb_last_seen', 'signature', 'file_type', 'reporter', 'origin_country', 'first_report', 'time_difference', 'time_difference(sec)', 'reference']


# To read saved local copy of database
df = pd.read_csv('/content/drive/MyDrive/Twitter_Work_Code/TIP/MB/mb_output_preprocessed_11_7_2022.csv',names=fields_name, encoding='utf-8', low_memory=False,lineterminator='\n')
# Filtering hashes
"""df = df[(df['ioc_type'] == 'hash')].reset_index(drop=True)
df=df[1:20]"""

# To collect the values from malwarebazar and update the dataframe
def report(index):
    ioc_type=df.loc[index,'ioc_type']
    if ioc_type not in ['hash']:
      return
    ioc = (df.loc[index,'ioc_value']).strip()

    # To get the api response   
    while True:
        try:
            response = requests.post("https://mb-api.abuse.ch/api/v1/",{'query':'get_info','hash':ioc})
            break
        except Exception as e:
            print({'Error':str(e)})

    response=response.json()
    print(response['query_status'])

    # Writing the data to dataframe
    if response['query_status'] == 'hash_not_found':
        df.loc[index, 'mb_status'] = "Not Found"
    else:
        df.loc[index, 'mb_status'] = "Malicious"
        print(df.loc[index, 'mb_status'])
        print(response['data'][0]['first_seen'])
        print(type(response['data'][0]['first_seen']))
        source_date = df.loc[index, 'tweet_date']
        tweet_date=source_date[:19]
        print('tweet type',type(tweet_date))
        first_seen_date = response['data'][0]['first_seen']
        print('tweet',tweet_date)
        print('mbdate',first_seen_date)
        tweet_date = datetime.strptime(tweet_date,'%Y-%m-%d %H:%M:%S')
        print('tweet type',type(tweet_date))
        first_seen_date = datetime.strptime(first_seen_date,'%Y-%m-%d %H:%M:%S')
        print("fdhhhhhhh",type(first_seen_date))
        if first_seen_date < tweet_date:
            df.loc[index, "first_report"] = "Malware Bazaar"
            time_diff = tweet_date-first_seen_date
            print("mb")
        else:            
            df.loc[index, 'first_report'] = "Twitter"
            time_diff = first_seen_date-tweet_date
            print("twitter")
        print(time_diff)
        print(df.loc[index, 'first_report'])
        df.loc[index, "time_difference"] = time_diff
        df.loc[index, "time_difference(sec)"] = time_diff.total_seconds()
        df.loc[index, "mb_first_seen"] = response['data'][0]['first_seen']
        df.loc[index, "mb_last_seen"] = response['data'][0]['last_seen']
        df.loc[index, "signature"] = response['data'][0]['signature']
        df.loc[index, 'file_type'] = response['data'][0]['file_type']
        df.loc[index, 'reporter'] = response['data'][0]['reporter']
        df.loc[index, 'origin_country'] = ISO3166[response['data'][0]['origin_country']]

        df.loc[index, 'reference'] = "https://bazaar.abuse.ch/sample/" + response['data'][0]['sha256_hash']
        print("test")
        print("Differebce",df.loc[index, 'time_difference'])
        print("test_sec", time_diff.total_seconds())
        print(df.loc[index, 'time_difference(sec)'])
    print(index, ioc, df.loc[index, "mb_status"])
# To run the queries as multithreaded for faster execution rate
with ThreadPoolExecutor(max_workers=1) as executor:
    executor.map(report, range(1,20))
    executor.shutdown(wait=True)

# Writeout the database as csv for storage
df.to_csv('/content/drive/MyDrive/Twitter_Work_Code/TIP/MB/MB_Output.csv', na_rep='', index=None,errors='ignore')

print(f'\nTime : {time.time() - start_time : .2f}')

hash_not_found
1 3f54d9331db97a9e41acf56ab52f0c8a2702b4eca5233cb5e2056080fcf47cd4 Not Found
hash_not_found
2 a1131f1e5dcdd983fa79308ecd868165a133f56708fcb0a9e1723459b9136c3b Not Found
hash_not_found
3 a5bfc9e07964f8dddeb95fc584cd965d Not Found
hash_not_found
4 2ec0540b5f51d63864bda26ab5f54b1fed06eab5f42c60b7c4a14152ddb456bf Not Found
hash_not_found
5 35cafdb930cf6cd24b65e5e639c3253c Not Found
hash_not_found
6 c136b1467d669a725478a6110ebaaab3cb88a3d389dfa688e06173c066b76fcf Not Found
ok
Malicious
2022-07-20 05:56:18
<class 'str'>
tweet type <class 'str'>
tweet 2021-01-02 06:10:02
mbdate 2022-07-20 05:56:18
tweet type <class 'datetime.datetime'>
fdhhhhhhh <class 'datetime.datetime'>
twitter
563 days, 23:46:16
Twitter
hash_not_found
8 da8591758cf7f86fd5bafacb7ce10c2f0b282aa8793227ab0381e13e1273b70f Not Found
hash_not_found
9 545360f94e9d827e1757e8d4768c3b652a55061216a2fe0894043e6850467eea Not Found
hash_not_found
10 6ea9ab1baa0efb9e19094440c317e21b Not Found
hash_not_found
11 fbd7939d6749

In [3]:
df

,tweet_date,account,ioc_type,ioc_value,type_of_attack,tweet_url,text,mb_status,mb_first_seen,mb_last_seen,signature,file_type,reporter,origin_country,first_report,time_difference,time_difference(sec),reference
0,tweet_date,account,ioc_type,ioc_value,type_of_attack,tweet_url,text,mb_status,mb_first_seen,mb_last_seen,signature,file_type,reporter,origin_country,first_report,time_difference,time_difference(sec),reference
1,2021-01-01 05:53:40+00:00,ReBensk,hash,3f54d9331db97a9e41acf56ab52f0c8a2702b4eca5233c...,['malware'],1.3448844755246162e+18,#Android #Banking #Trojan #Malware\n@malwrhunt...,Not Found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-01 08:20:02+00:00,HeliosCert,hash,a1131f1e5dcdd983fa79308ecd868165a133f56708fcb0...,['malware'],1.3449213105022648e+18,Sample submitted\n2021-01-01 08:20:02\nDionaea...,Not Found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-01 08:56:21+00:00,ecarlesi,hash,a5bfc9e07964f8dddeb95fc584cd965d,['phishing'],1.344930451694076e+18,Threat on hxxps://check-payeeid[.]com/HSBC/idv...,Not Found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-01 11:35:02+00:00,HeliosCert,hash,2ec0540b5f51d63864bda26ab5f54b1fed06eab5f42c60...,['malware'],1.3449703824204186e+18,Sample submitted\n2021-01-01 11:35:01\nURL: ht...,Not Found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4541,2021-07-31 12:30:02+00:00,HeliosCert,hash,02781974804e2467b3900993e48cb5b884bd68f03d0045...,['malware'],1.4214480652617073e+18,Sample submitted\n2021-07-31 12:30:02\nDionaea...,Clean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4542,2021-07-31 16:58:04+00:00,MalwarePatrol,hash,fbe51695e97a45dc61967dc3241a37dc,['malware'],1.4215155191155302e+18,One of the active #malware samples detected on...,Malicious,2021-03-01 23:09:40,2022-03-06 17:31:04,NaN,elf,realTWISTEDCATS,NaN,NaN,NaN,NaN,NaN
4543,2021-07-31 16:58:04+00:00,MalwarePatrol,hash,1ed14334b5b71783cd6ec14b8a704fe48e600cf0,['malware'],1.4215155191155302e+18,One of the active #malware samples detected on...,Malicious,2021-03-01 23:09:40,2022-03-06 17:31:04,NaN,elf,realTWISTEDCATS,NaN,NaN,NaN,NaN,NaN
4544,2021-07-31 17:15:02+00:00,HeliosCert,hash,c359b6169a1e9eeae664a5811ebc8512dd224e6930613b...,['malware'],1.421519789349118e+18,Sample submitted\n2021-07-31 17:15:02\nDionaea...,Clean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Twitter_Work_Code/TIP/MB/mb_output.csv',encoding='utf-8', low_memory=False,lineterminator='\n')
# Filtering hashesv

In [ ]:
df

,tweet_date,account,ioc_type,ioc_value,type_of_attack,tweet_url,text,mb_status,mb_first_seen,mb_last_seen,signature,file_type,reporter,origin_country,first_report,time_difference,time_difference(sec),reference
0,tweet_date,account,ioc_type,ioc_value,type_of_attack,tweet_id,text,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-01 00:09:03+00:00,MalwarePatrol,url,http://dubioustimes.com/,['phishing'],1.3447977507481887e+18,Another active #Phishing targeting Google Driv...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-01 00:09:03+00:00,MalwarePatrol,url,https://t.co/m4mjDMt7az,['phishing'],1.3447977507481887e+18,Another active #Phishing targeting Google Driv...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-01 00:09:03+00:00,MalwarePatrol,domain,dubioustimes.com,['phishing'],1.3447977507481887e+18,Another active #Phishing targeting Google Driv...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-01 00:09:03+00:00,MalwarePatrol,domain,t.co,['phishing'],1.3447977507481887e+18,Another active #Phishing targeting Google Driv...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93790,2021-07-31 21:01:16+00:00,ShoestringSoc,ip,197.156.73.150,['honeypot'],1.4215767218080522e+18,New attacker reported:\n\nIP: 197.156.73.150...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93791,2021-07-31 21:19:38+00:00,ecarlesi,url,http://xmyparcel21-redelivery.com/postoffice/fee/,['phishing'],1.4215813444679803e+18,Threat on hxxps://xmyparcel21-redelivery[.]com...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93792,2021-07-31 21:25:08+00:00,ecarlesi,url,http://igbusiness-form.com/283772919101/,['phishing'],1.421582728231678e+18,Threat on hxxps://igbusiness-form[.]com/283772...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93793,2021-07-31 22:56:45+00:00,phishunt_io,ip,217.160.0.240,['phishing'],1.421605783297413e+18,#NewPhishing | #phishing #scam\n\n🌐 /bankofame...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
mb_df=pd.read_csv("/content/drive/MyDrive/TIP/MB/MB_Output.csv",lineterminator='\n')
mb_df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,tweet_date,account,ioc_type,ioc_value,type_of_attack,tweet_url,text,mb_status,mb_first_seen,mb_last_seen,signature,file_type,reporter,origin_country,first_report,time_difference,time_difference(sec),reference
0,tweet_date,account,ioc_type,ioc_value,type_of_attack,tweet_id,text,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-01 00:00:02+00:00,ItsTrigger,url,https://pastebin.com/raw/yS4QMNxk,[],1344795482804117504,2020 was rough for us all. I don't usually wri...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-01 00:00:26+00:00,Paula_Piccard,url,https://zd.net/2WXvcDR,"['malware', 'phishing']",1344795582053937153,SolarWinds hackers accessed Microsoft source c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-01 00:03:25+00:00,MoustachedTrex,url,https://pastebin.com/Wdgiq6Mm,[],1344796334268805125,https://t.co/L4NQPYJ8K7 A list of everything I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-01 00:06:06+00:00,An0n_Truth,url,https://paper.li/An0n_Truth/1357567312?edition...,['malware'],1344797006288584705,The latest The An0nym0us Truth! https://t.co/X...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355402,2021-07-31 23:50:21+00:00,PMProuk,url,https://www.canadianunderwriter.ca/insurance/a...,['cyberattack'],1421619269947822080,"As cyberattacks skyrocket, Canada needs to wor...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355403,2021-07-31 23:52:04+00:00,cyberinform,url,https://bit.ly/3j7xo6w,['ransomware'],1421619701889785861,#Ransomware attacks in #2021 have already surp...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355404,2021-07-31 23:52:04+00:00,cyberinform,domain,bit.ly,['ransomware'],1421619701889785861,#Ransomware attacks in #2021 have already surp...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355405,2021-07-31 23:52:10+00:00,CyberSecDN,domain,marketresearchtelecast.com,['malware'],1421619729651748865,Security researcher: Criminals use Discord to ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df[19230:20000]

,tweet_date,account,ioc_type,ioc_value,type_of_attack,tweet_url,text,mb_status,mb_first_seen,mb_last_seen,signature,file_type,reporter,origin_country,first_report,time_difference,time_difference(sec),reference
19230,2021-07-22 05:06:38+00:00,bamitav,url,https://www.murfreesboropost.com/news/national...,"['malware', 'ransomware', 'botnet', 'cyberatta...",1418074989840846848,"#US blames #China for hacks, opening new front...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19231,2021-07-22 05:06:38+00:00,bamitav,domain,murfreesboropost,"['malware', 'ransomware', 'botnet', 'cyberatta...",1418074989840846848,"#US blames #China for hacks, opening new front...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19232,2021-07-22 05:05:28+00:00,MoHossain,url,https://www.ft.com/content/76fdac7c-7076-47a4-...,"['ransomware', 'cyberattack']",1418074693987405827,Exporting Chinese surveillance: the security r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19233,2021-07-22 05:04:52+00:00,GrimroPOE,url,https://pastebin.com/nxRqS0hV,[],1418074545156546560,@Zizaran Definitely a SC build but outside of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19234,2021-07-22 05:01:13+00:00,ShoestringSoc,ip,139.0.102.26,['honeypot'],1418073627308658689,New attacker reported:\n\nIP: 139.0.102.26\n...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2021-07-21 17:09:14+00:00,blumirasec,url,https://www.blumira.com/sam-database-vulnerabi...,['cve'],1417894449166888966,🚨 Update🚨 Blumira's security team has been wor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19996,2021-07-21 17:09:14+00:00,blumirasec,domain,blumira,['cve'],1417894449166888966,🚨 Update🚨 Blumira's security team has been wor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19997,2021-07-21 17:09:04+00:00,vigilance_en,url,https://vigilance.fr/vulnerability/NVIDIA-Grap...,['cve'],1417894405881663489,Vigil@nce #Vulnerability of NVIDIA Graphics Dr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19998,2021-07-21 17:09:04+00:00,vigilance_en,CVE,CVE-2021-1093,['cve'],1417894405881663489,Vigil@nce #Vulnerability of NVIDIA Graphics Dr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
